In [6]:
import os
import re
import json
from PyPDF2 import PdfReader
from datetime import datetime

def extract_text_from_pdf(pdf_path):
    """
    Extrait le texte d'un fichier PDF.
    """
    reader = PdfReader(pdf_path)
    text = ""
    for page in reader.pages:
        text += page.extract_text()
    return text

def is_holiday_or_weekend(date_str):
    """
    Vérifie si une date donnée tombe un jour férié ou un week-end.
    Exemple simple à adapter selon le contexte.
    """
    try:
        date = datetime.strptime(date_str, "%d %B %Y")  # Exemple : "15 février 2025"
        # Vérifier les week-ends
        if date.weekday() >= 5:  # 5 = samedi, 6 = dimanche
            return True

        # Vérifier les jours fériés (ajoutez vos propres dates fériées ici)
        holidays = [
            "01 January 2025", "14 July 2025", "25 December 2025"
        ]
        if date.strftime("%d %B %Y") in holidays:
            return True
    except ValueError:
        pass
    return False

def detect_vices(text):
    """
    Détecte les vices de procédure basés sur des règles simples.
    Retourne une liste vide si aucun vice n'est détecté.
    """
    vices = []

    # Absence de signature
    if not re.search(r"(signature|signé par)", text, re.IGNORECASE):
        vices.append("Absence de signature : La notification ne comporte pas la signature de l’huissier.")

    # Erreur de date
    date_match = re.search(r"\d{1,2} \w+ \d{4}", text)
    if date_match:
        date_str = date_match.group(0)
        if is_holiday_or_weekend(date_str):
            vices.append(f"Erreur de date : La date mentionnée ({date_str}) tombe un jour férié ou non ouvré.")

    # Absence de preuve de notification
    if not re.search(r"(accusé de réception|preuve de remise|huissier)", text, re.IGNORECASE):
        vices.append("Absence de preuve de notification : Le courrier ne mentionne pas si une preuve de remise sera utilisée.")

    return vices  # Si la liste est vide, aucun vice n'a été détecté


def annotate_text(text, vices):
    """
    Pré-anote le texte avec les informations importantes et les vices détectés.
    """
    annotations = []

    # Extractions simples (dates, parties, numéro de dossier)
    date_match = re.search(r"\d{1,2} \w+ \d{4}", text)
    if date_match:
        annotations.append({"start": date_match.start(), "end": date_match.end(), "label": "DATE_EMISSION"})

    party_match = re.search(r"(?<=Partie concernée : ).+", text)
    if party_match:
        annotations.append({"start": party_match.start(), "end": party_match.end(), "label": "PARTIE_CONCERNEE"})

    dossier_match = re.search(r"(?<=Numéro de dossier : )\d+", text)
    if dossier_match:
        annotations.append({"start": dossier_match.start(), "end": dossier_match.end(), "label": "NUMERO_DOSSIER"})

    # Ajout des vices détectés
    for vice in vices:
        annotations.append({"label": "VICE_NOTIFICATION", "description": vice})

    return annotations

def process_pdfs_in_directory(directory_path, output_json):
    """
    Traite tous les fichiers PDF dans un répertoire et exporte les annotations en JSON.
    """
    results = []

    # Lister tous les fichiers PDF dans le répertoire
    for filename in os.listdir(directory_path):
        if filename.endswith(".pdf"):
            file_path = os.path.join(directory_path, filename)
            print(f"Traitement de : {file_path}")

            # Extraction du texte
            text = extract_text_from_pdf(file_path)

            # Détection des vices
            vices = detect_vices(text)

            # Annotation du texte
            annotations = annotate_text(text, vices)

            # Ajouter au résultat
            annotations = annotate_text(text, vices)
            label = 1 if vices else 0  # 1 = avec vice, 0 = sans vice
            results.append({
              "file_name": filename,
              "text": text,
              "annotations": annotations,
              "vices_detected": vices,
              "label": label
            })


    # Exporter les résultats en JSON
    with open(output_json, "w", encoding="utf-8") as json_file:
        json.dump(results, json_file, ensure_ascii=False, indent=4)

    print(f"Annotations exportées dans : {output_json}")

# Utilisation du script
if __name__ == "__main__":
    input_directory = "Notifications/PasVice"  # Remplacez par le chemin de votre répertoire contenant les PDF
    output_json = "Notifications/annotations_results.json"  # Chemin du fichier JSON de sortie
    process_pdfs_in_directory(input_directory, output_json)


Traitement de : Notifications/PasVice/Notification4Vice.pdf
Traitement de : Notifications/PasVice/Notification6Vice.pdf
Traitement de : Notifications/PasVice/Notification5.pdf
Traitement de : Notifications/PasVice/Notification1.pdf
Traitement de : Notifications/PasVice/Notification3Vice.pdf
Traitement de : Notifications/PasVice/Notification2.pdf
Annotations exportées dans : Notifications/annotations_results.json


In [15]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
import torch
from sklearn.model_selection import train_test_split

# Charger les données
def load_data(json_path):
    import json
    with open(json_path, "r", encoding="utf-8") as f:
        data = json.load(f)
    return data

# Charger le fichier JSON avec vos données
data = load_data("Notifications/annotations_results.json")  # Remplacez par le chemin de votre fichier JSON

# Créer un Dataset Hugging Face
texts = [item["text"] for item in data]
labels = [item["label"] for item in data]

# Séparer en jeu d'entraînement et de validation
train_texts, val_texts, train_labels, val_labels = train_test_split(texts, labels, test_size=0.2)

# Créer des datasets
train_dataset = Dataset.from_dict({"text": train_texts, "label": train_labels})
val_dataset = Dataset.from_dict({"text": val_texts, "label": val_labels})

# Charger CamemBERT
model_name = "camembert-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

# Tokenizer les données
def tokenize_function(example):
    return tokenizer(example["text"], padding="max_length", truncation=True)

train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)

# Spécifier les arguments d'entraînement
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_dir="./logs",
    load_best_model_at_end=True,
    logging_steps=10
)

# Définir un Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer
)

# Entraîner le modèle
trainer.train()

# Sauvegarder le modèle entraîné
model.save_pretrained("./trained_model")
tokenizer.save_pretrained("./trained_model")


Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at camembert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/4 [00:00<?, ? examples/s]

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.26.0`: Please run `pip install transformers[torch]` or `pip install 'accelerate>={ACCELERATE_MIN_VERSION}'`